In [3]:
import math
import numpy as np

# Raw data


In [19]:
D = 25e-3 # m
L = 1.65 # m
mu = 8.9e-4 # Pa*s
rho = 1000 # kg/m^3

H_corr = 6.38 - 4.28


# Experimental data

In [18]:

H1 = np.array([5.15, 4.75, 3.32, 4.44, 5.74, 4.58, 5.38, 4.68, 7.36]) # mm
H2 = np.array([3.18, 4.48, 3.44, 4.86, 7.36, 8.34,10.68, 17.25, 28.52]) # mm

vol = np.array([0.000353, 0.000767, 0.00118, 0.00148, 0.00212, 0.00248, 0.00365, 0.00528, 0.00775]) # m^3
t = np.array([53.73, 34.25, 32.96, 35.14, 34.75, 38.26, 32.75, 33.45, 36.15]) # s

In [16]:
Q = vol / t
print(f'Volumetric flow rate: {Q} m^3/s')

area = math.pi * D**2 / 4
V = Q / area
print(f'Velocity: {V} m/s')


Volumetric flow rate: [6.56988647e-06 2.23941606e-05 3.58009709e-05 4.21172453e-05
 6.10071942e-05 6.48196550e-05 1.11450382e-04 1.57847534e-04
 2.14384509e-04] m^3/s
Velocity: [0.01338406 0.04562101 0.07293314 0.08580055 0.12428284 0.13204952
 0.22704485 0.32156435 0.43674054] m/s


In [17]:
N_Re = rho * V * D / mu
print(f"Reynold's number: {N_Re}")


Reynold's number: [  375.95682055  1281.48902592  2048.68367894  2410.12774099
  3491.09088654  3709.25609043  6377.66441483  9032.70660043
 12267.99256742]


In [ ]:
H_2_corr = H2 + H_corr
delta_H_corr = H_2_corr - H1

g = 9.81 # m/s^2

delta_P = rho * g * delta_H_corr